In [3]:
from openai import AsyncOpenAI

client = AsyncOpenAI(api_key="1989", base_url="http://localhost:8000/v1")

completion = await client.chat.completions.create(
    model="deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B",
    messages=[{"role": "user", "content": "What is the capital of France?"}],
)

completion.choices[0].message.content

'<think>\n\n</think>\n\nThe capital of France is Paris.'

In [9]:
import json

puzzles = json.load(open("./data/temporal-clue-puzzles.json"))

In [23]:
chat_completion = await client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": "What is the capital of France?",
        }
    ],
    model="deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B",
    logprobs=True
)

chat_completion.choices[0].logprobs

ChoiceLogprobs(content=[ChatCompletionTokenLogprob(token='token_id:151648', bytes=[60, 116, 104, 105, 110, 107, 62], logprob=0.0, top_logprobs=[]), ChatCompletionTokenLogprob(token='token_id:271', bytes=[10, 10], logprob=-0.6931719779968262, top_logprobs=[]), ChatCompletionTokenLogprob(token='token_id:151649', bytes=[60, 47, 116, 104, 105, 110, 107, 62], logprob=-7.986990567587782e-06, top_logprobs=[]), ChatCompletionTokenLogprob(token='token_id:271', bytes=[10, 10], logprob=-9.298280929215252e-06, top_logprobs=[]), ChatCompletionTokenLogprob(token='token_id:785', bytes=[84, 104, 101], logprob=-0.006816348992288113, top_logprobs=[]), ChatCompletionTokenLogprob(token='token_id:6722', bytes=[32, 99, 97, 112, 105, 116, 97, 108], logprob=-0.0011029120068997145, top_logprobs=[]), ChatCompletionTokenLogprob(token='token_id:315', bytes=[32, 111, 102], logprob=-0.00015698630886618048, top_logprobs=[]), ChatCompletionTokenLogprob(token='token_id:9625', bytes=[32, 70, 114, 97, 110, 99, 101], log

In [24]:
[
    token_logprob
    for token_logprob in chat_completion.choices[0].logprobs.content
    if token_logprob.bytes == [60, 47, 116, 104, 105, 110, 107, 62]
]

[ChatCompletionTokenLogprob(token='token_id:151649', bytes=[60, 47, 116, 104, 105, 110, 107, 62], logprob=-7.986990567587782e-06, top_logprobs=[])]

In [31]:
async for chunk in await client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": sorted(puzzles, key=lambda x: x["num_clues"])[0]["prompt"],
        }
    ],
    model="deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B",
    stream=True,
    # frequency_penalty=0.1,
    # presence_penalty=0.1,
    logit_bias={151649: 8},
):
    print(chunk.choices[0].delta.content, end="", flush=True)

<think>
Okay, so I'm trying

 to solve this murder mystery. Let's see, there are two suspects: Mr. Green and Colonel Mustard. Weapons found are a knife and a lead pipe. The murder happened in one of two rooms: Gazebo or Trophy Room. The victim, Mr. Boddy, is found dead, and the room he was in must have both the weapon and the suspect. 

First, I'll try to figure out the layout. The rooms are laid out in a grid, which I can visualize as three rows. Let me represent them as rows from top to bottom:

Row 1: N (Maybe North disappears from 2D or is Maybe room numbering is different? Wait, looking back at the layout:

N  
W 1 E  
W 2 E  
  S  

So it's a 3-row structure? First row has room N, then two rooms W1 and W2 on the same row, then W3 and S on the third row? Wait, maybe it's better to parse the layout again.

Wait, the layout is a bit unclear. The rows are row 1, which has room N. Then rows 2 and 3 are presented with W1, W2, W3 on the bottom? Hmm, maybe it's more like:

N  
W1 E  
W2 E  
  S  

Wait, that doesn't

In [ ]:
import atreides

model = atreides.train_model(
    base_model="deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B",
    train_dataset="capitals/train",
    val_dataset="capitals/val",
)

In [ ]:

from prisma import Prisma

client = AsyncOpenAI()
prisma = Prisma()

await prisma.connect()

datasets = await prisma.dataset.find_many()

print(datasets)

[]


In [4]:
import json

dataset = await prisma.dataset.create(
    {
        "name": "capitals/train",
        "prompts": {
            "create": [
                {
                    "messages": json.dumps(
                        [{"role": "user", "content": "What is the capital of France?"}]
                    ),
                    "metadata": json.dumps({})
                }
            ]
        },
    }
)
dataset

Dataset(id=1, name='capitals/train', prompts=None)

In [8]:
completion = await prisma.completion.create(
    {
        "prompt_id": 1,
        "model": "gpt-4o",
        "results": json.dumps([{"role": "assistant", "content": "Paris"}]),
    }
)
completion

Completion(id=1, prompt_id=1, prompt=None, model='gpt-4o', results='[{"role": "assistant", "content": "Paris"}]', rewards=[Reward(id=1, completion_id=1, completion=None, title='Accuracy', description='1.0 if the capital is correct, 0.0 otherwise', value=1.0, weight=1.0)])

In [7]:
dataset = await prisma.dataset.find_first(include={"prompts": True})
dataset.prompts

[Prompt(id=1, messages='[{"role": "user", "content": "What is the capital of France?"}]', metadata='{}', datasets=None, completions=None)]

In [3]:
import atreides

try:
    train_dataset = await atreides.get_dataset("capitals/train")
except atreides.NotFoundError:
    train_dataset = await atreides.create_dataset(
        name="capitals/train",
        tasks=[
            {
                "messages": [
                    {"role": "user", "content": "What is the capital of France?"}
                ],
                "pattern_rewards": [
                    {
                        "regex_pattern": r"The capital of France is (\w+)",
                        "expected_capture_groups": ["Paris"],
                    },
                ],
            },
        ],
    )

try:
    val_dataset = await atreides.get_dataset("capitals/val")
except atreides.NotFoundError:
    val_dataset = await atreides.create_dataset(
        name="capitals/val",
        tasks=[
            {
                "messages": [
                    {"role": "user", "content": "What is the capital of England?"}
                ],
                "pattern_rewards": [
                    {
                        "regex_pattern": r"The capital of England is (\w+)",
                        "expected_capture_groups": ["London"],
                    },
                ],
            },
        ],
    )